<a href="https://colab.research.google.com/github/tkdlqh2/wanted_8/blob/main/3_NLU_%EB%AC%B8%EC%9E%A5%EC%9C%A0%EC%82%AC%EB%8F%84%EA%B3%84%EC%82%B0/NLU_sentence_klue_roberta_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 설정 및 모듈 임포트

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install transformers

In [6]:
import os
import sys
import pandas as pd
import numpy as np 

import re
import requests

import torch
import torch.nn as nn
import tensorflow as tf

import math
import logging
from datetime import datetime

from torch.utils.data import DataLoader
#from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [7]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [8]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
# 전처리된 데이터 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/data/klue_sts_preprocessed.csv')

In [63]:
# 일단 val 데이터 불러오기
val_df = pd.read_json('/content/drive/MyDrive/data/klue-sts-v1.1/klue-sts-v1.1_dev.json')


In [11]:
train_df.head(1).T

,0
guid,klue-sts-v1_train_00000
source,airbnb-rtt
sentence1,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
sentence2,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.
labels,"{'label': 3.7, 'real-label': 3.714285714285714..."
annotations,"{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
binary,1
spell_checked1,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다
spell_checked2,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다
khaii1,숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다


In [12]:
# 중복행 발견
train_df.duplicated(['sentence1', 'sentence2']).sum()

7

In [13]:
# sentence1, sentence2, annotations 동일한 데이터 발견 
# 하나씩만 남기고 제거
train_df[train_df.duplicated(['sentence1','sentence2', 'annotations'], keep=False)]

,guid,source,sentence1,sentence2,labels,annotations,binary,spell_checked1,spell_checked2,khaii1,khaii2
1514,klue-sts-v1_train_01514,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['0...",1,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사는 지역에서 추진 ...,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사가 지역에서 추진 ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...
1661,klue-sts-v1_train_01661,airbnb-rtt,택시타고 공항갔을 때 20유로로 15분내에 도착했었어요.,"택시를 타고 공항에 갔을 때, 15분만에 20유로에 도착했습니다.","{'label': 4.7, 'real-label': 4.666666666666667...","{'agreement': '0:0:0:0:2:4', 'annotators': ['1...",1,택시 타고 공항 갔을 때 유로로 분 내에 도착했었어요,택시를 타고 공항에 갔을 때 분 만에 유로에 도착했습니다,택시 타 고 공항 가 았 을 때 유로 분 도착 하 았었 어요,택시 타 고 공항 가 았 을 때 분 유로 도착 하 였 습니다
1715,klue-sts-v1_train_01715,airbnb-rtt,택시타고 공항갔을 때 20유로로 15분내에 도착했었어요.,"택시를 타고 공항에 갔을 때, 15분만에 20유로에 도착했습니다.","{'label': 4.7, 'real-label': 4.666666666666667...","{'agreement': '0:0:0:0:2:4', 'annotators': ['1...",1,택시 타고 공항 갔을 때 유로로 분 내에 도착했었어요,택시를 타고 공항에 갔을 때 분 만에 유로에 도착했습니다,택시 타 고 공항 가 았 을 때 유로 분 도착 하 았었 어요,택시 타 고 공항 가 았 을 때 분 유로 도착 하 였 습니다
3872,klue-sts-v1_train_03872,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1...",1,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사는 지역에서 추진 ...,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사가 지역에서 추진 ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...
5139,klue-sts-v1_train_05139,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.6, 'real-label': 4.571428571428571...","{'agreement': '0:0:0:0:3:4', 'annotators': ['0...",1,지난달 하루 평균 수출액이 전년대비 증가한 가운데 자동차는 년 월 이후 처음으로 ...,지난달 평균 일일 수출액이 전년 동월 대비 증가하면서 자동차의 수출은 년 월 이후...,지난달 하루 평균 수출액 전년 대비 증가 하 ㄴ 가운데 자동차 이후 처음 수출액 억...,지난달 평균 일일 수출액 전년 동월 대비 증가 하 면서 자동차 수출 이후 처음 억 ...
5292,klue-sts-v1_train_05292,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['0...",1,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사는 지역에서 추진 ...,제차 전략회의에서 대전 경기 강원 전남 제주 경남 등 개 시도지사가 지역에서 추진 ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...,제차 전략 회의 대전 경 기 강원 전남 제주 경남 개 시도 지사 지역 추진 이 ㄴ ...
7045,klue-sts-v1_train_07045,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.0, 'real-label': 4.0, 'binary-labe...","{'agreement': '0:0:0:1:5:1', 'annotators': ['1...",1,지난달 하루 평균 수출액이 전년대비 증가한 가운데 자동차는 년 월 이후 처음으로 ...,지난달 평균 일일 수출액이 전년 동월 대비 증가하면서 자동차의 수출은 년 월 이후...,지난달 하루 평균 수출액 전년 대비 증가 하 ㄴ 가운데 자동차 이후 처음 수출액 억...,지난달 평균 일일 수출액 전년 동월 대비 증가 하 면서 자동차 수출 이후 처음 억 ...
10908,klue-sts-v1_train_10908,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.0, 'real-label': 4.0, 'binary-labe...","{'agreement': '0:0:0:1:5:1', 'annotators': ['1...",1,지난달 하루 평균 수출액이 전년대비 증가한 가운데 자동차는 년 월 이후 처음으로 ...,지난달 평균 일일 수출액이 전년 동월 대비 증가하면서 자동차의 수출은 년 월 이후...,지난달 하루 평균 수출액 전년 대비 증가 하 ㄴ 가운데 자동차 이후 처음 수출액 억...,지난달 평균 일일 수출액 전년 동월 대비 증가 하 면서 자동차 수출 이후 처음 억 ...
10939,klue-sts-v1_train_10939,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.6, 'real-label': 4.571428571428571...","{'agreement': '0:0:0:0:3:4', 'annotators': ['0...",1,지난달 하루 평균 수출액이 전년대비 증가한 가운데 자동차는 년 월 이후 처음으로 ...,지난달 평균 일일 수출액이 전년 동월 대비 증가하면서 자동차의 수출은 년 월 이후...,지난달 하루 평균 수출액 전년 대비 증가 하 ㄴ 가운데 자동차 이후 처음 수출액 억...,지난달 평균 일일 수출액 전년 동월 대비 증가 하 면서 자동차 수출 이후 처음 억 ...
11112,klue-sts-v1_train_11112,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남

In [14]:
train_df = train_df.drop_duplicates(['sentence1','sentence2'], keep='first')

In [15]:
# 확인
train_df.duplicated(['sentence1', 'sentence2']).sum()

0

### 1. Baseline Model - Sentence Transformer 사용

In [16]:
# 학습 경과 지켜보기 위해 logger 초기화
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [17]:
# 사전학습 언어모델
model_name = "klue/roberta-large"

In [18]:
# Hyperparameter
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [19]:
# 사전학습 언어모델을 임베딩을 추출할 Embedder로 사용
embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [20]:
# Pooler > Embedder 에서 추출된 토큰 단위 임베딩들을 가지고 문장 임베딩을 어떻게 계산할 것인지를 결정
# Mean pooling
# 모델이 반환한 모든 토큰 임베딩을 더하고 그 개수만큼 나누어 문장을 대표하는 임베딩으로 사용
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [21]:
# embedder, pooler 모듈로 구성된 모델 정의
model = SentenceTransformer(modules=[embedding_model, pooler])

2022-03-17 11:58:27 - Use pytorch device: cuda


In [24]:
# 학습에 사용할 칼럼만 덜어내기
# 이번 학습에선 khaiii 형태소변환 된 데이터
datasets = train_df[['guid','labels','khaii1','khaii2']]
datasets.head()

,guid,labels,khaii1,khaii2
0,klue-sts-v1_train_00000,"{'label': 3.7, 'real-label': 3.714285714285714...",숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다,숙박 시설 위치 쉽 게 찾 을 있 고 한국 대표 적 이 ㄴ 반지하 숙박 시설 이 ㅂ니다
1,klue-sts-v1_train_00001,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...",위반 행위 조사 거부 방해기피 하 ㄴ 만 이하 과태료 부과 대상 이 다,시민 들 스스로 자발 적 이 ㄴ 예방 노력 하 ㄴ 아 산 아니 었 다
2,klue-sts-v1_train_00002,"{'label': 0.30000000000000004, 'real-label': 0...",회사 보내 ㄴ 메일 이 지메일 아니 라 다른 지메일 계정 전달 하 여 주 어,사람 들 주로 네이버 메일 쓰 는 이유 알리 어 주 어
3,klue-sts-v1_train_00003,"{'label': 0.6000000000000001, 'real-label': 0....",긴급 고용 안정 지원금 지역 고용 대응 특별 지원금 지자 체 별 소상 공 이 ㄴ 지...,고용 보험 차 아 고용 안 전망 이 라면 국민 취업 지원 제도 차 고용 안 전망 이...
4,klue-sts-v1_train_00004,"{'label': 4.7, 'real-label': 4.714285714285714...",호스트 답장 늦 으나 개선 되 ㄹ 보이 ㅂ니다,호스트 응답 늦 었 지만 개선 되 ㄹ 보이 ㅂ니다


In [59]:
# str을 dict로 바꾸는 코드... 왜안됐던거지
for i in range(len(datasets)):
  example = datasets.iloc[i]
  print(eval(example['labels'])['label'])
  break

3.7


In [61]:
# 데이터셋을 sentence-transformer input형식에 맞춰 변환 (InputExample)
# score > 0점에서 5점 사이의 값, normalize 통해 0 < x < 1 값으로 정규화 (일반적)

train_samples = []
dev_samples = [] #train을 train/val로 나누고
test_samples = [] #val을 test로 검증

# KLUE STS 내 훈련 데이터 InputExample 객체로 변환
for i in range(len(datasets)):
  example = datasets.iloc[i]
  score = float(eval(example["labels"])["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화
  inp_example = InputExample(
      texts=[example["khaii1"], example["khaii2"]],
      label=score
  )
  if int(example['guid'][-1:]) % 9 == 0:  # 랜덤 샘플링 해야되는데 일단 걍 이렇게
    dev_samples.append(inp_example)
  else:
    train_samples.append(inp_example)

In [62]:
# 4 : 1
len(train_samples), len(dev_samples)

(9329, 2332)

In [64]:
val_df.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_dev_00000,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
1,klue-sts-v1_dev_00001,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,"{'label': 1.4, 'real-label': 1.428571428571429...","{'agreement': '0:4:3:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_dev_00002,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,"{'label': 1.3, 'real-label': 1.285714285714286...","{'agreement': '0:5:2:0:0:0', 'annotators': ['0..."
3,klue-sts-v1_dev_00003,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.","{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['1..."
4,klue-sts-v1_dev_00004,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,"{'label': 2.5, 'real-label': 2.5, 'binary-labe...","{'agreement': '1:0:1:3:1:0', 'annotators': ['0..."


In [66]:
# validation셋으로 test셋 만들기 (전처리 X)
for i in range(len(val_df)):
    example = val_df.iloc[i]
    score = float(example["labels"]["label"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [68]:
# 확인 (어 근데 특문 왜 안없어졌지?)
train_samples[0].texts, train_samples[0].label

(['위반 행위 조사 거부 방해기피 하 ㄴ 만 이하 과태료 부과 대상 이 다',
  '시민 들 스스로 자발 적 이 ㄴ 예방 노력 \xa0 하 ㄴ 아 산 아니 었 다'],
 0.0)

In [69]:
test_samples[0].texts, test_samples[0].label

(['무엇보다도 호스트분들이 너무 친절하셨습니다.', '무엇보다도, 호스트들은 매우 친절했습니다.'], 0.9800000000000001)

In [71]:
dev_samples[0].texts, dev_samples[0].label

(['숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다',
  '숙박 시설 위치 쉽 게 찾 을 있 고 한국 대표 적 이 ㄴ 반지하 숙박 시설 이 ㅂ니다'],
 0.74)

In [72]:
# DataLoader와 Loss 설정 (코사인유사도)
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [74]:
# Evaluator 정의
# 검증데이터를 활용하여 모델의 문장 임베딩 간 코사인 유사도가 얼마나 골드 라벨에 가까운지 계산하는 역할
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [75]:
# 모델 학습에 사용될 Warm up Steps 설정
# 예제에서 사용된 훈련 배치 수의 10% 값
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2022-03-17 12:53:42 - Warmup-steps: 117


In [76]:
# fit으로 모델 학습
# 훈련 과정을 통해 매 에폭 마다 얻어지는 체크포인트에 대해 
# Evaluator 가 학습된 모델의 코사인 유사도와 골드 라벨 간 피어슨, 스피어만 상관 계수를 계산해 기록

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/292 [00:00<?, ?it/s]

2022-03-17 12:58:34 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-03-17 12:58:46 - Cosine-Similarity :	Pearson: 0.9528	Spearman: 0.9042
2022-03-17 12:58:46 - Manhattan-Distance:	Pearson: 0.9427	Spearman: 0.9016
2022-03-17 12:58:46 - Euclidean-Distance:	Pearson: 0.9429	Spearman: 0.9024
2022-03-17 12:58:46 - Dot-Product-Similarity:	Pearson: 0.9441	Spearman: 0.8910
2022-03-17 12:58:46 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-17_11-55-25


Iteration:   0%|          | 0/292 [00:00<?, ?it/s]

2022-03-17 13:02:33 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2022-03-17 13:02:44 - Cosine-Similarity :	Pearson: 0.9540	Spearman: 0.9101
2022-03-17 13:02:44 - Manhattan-Distance:	Pearson: 0.9467	Spearman: 0.9074
2022-03-17 13:02:44 - Euclidean-Distance:	Pearson: 0.9469	Spearman: 0.9081
2022-03-17 13:02:44 - Dot-Product-Similarity:	Pearson: 0.9464	Spearman: 0.8958
2022-03-17 13:02:44 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-17_11-55-25


Iteration:   0%|          | 0/292 [00:00<?, ?it/s]

2022-03-17 13:06:34 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2022-03-17 13:06:46 - Cosine-Similarity :	Pearson: 0.9571	Spearman: 0.9129
2022-03-17 13:06:46 - Manhattan-Distance:	Pearson: 0.9477	Spearman: 0.9089
2022-03-17 13:06:46 - Euclidean-Distance:	Pearson: 0.9479	Spearman: 0.9098
2022-03-17 13:06:46 - Dot-Product-Similarity:	Pearson: 0.9501	Spearman: 0.9003
2022-03-17 13:06:46 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-17_11-55-25


Iteration:   0%|          | 0/292 [00:00<?, ?it/s]

2022-03-17 13:10:35 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2022-03-17 13:10:47 - Cosine-Similarity :	Pearson: 0.9577	Spearman: 0.9135
2022-03-17 13:10:47 - Manhattan-Distance:	Pearson: 0.9478	Spearman: 0.9094
2022-03-17 13:10:47 - Euclidean-Distance:	Pearson: 0.9481	Spearman: 0.9104
2022-03-17 13:10:47 - Dot-Product-Similarity:	Pearson: 0.9512	Spearman: 0.9023
2022-03-17 13:10:47 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-17_11-55-25


In [77]:
# 학습된 모델 테스트 하기
# Evaluator 초기화
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

2022-03-17 13:11:52 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-large-2022-03-17_11-55-25
2022-03-17 13:11:57 - Use pytorch device: cuda


In [78]:
# 테스트셋에 대해 각 상관계수 계산 
test_evaluator(model, output_path=model_save_path)

2022-03-17 13:12:06 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-17 13:12:11 - Cosine-Similarity :	Pearson: 0.8556	Spearman: 0.8537
2022-03-17 13:12:11 - Manhattan-Distance:	Pearson: 0.8484	Spearman: 0.8446
2022-03-17 13:12:11 - Euclidean-Distance:	Pearson: 0.8475	Spearman: 0.8439
2022-03-17 13:12:11 - Dot-Product-Similarity:	Pearson: 0.8361	Spearman: 0.8313


0.8537395988945635

In [ ]:
# 0.85로 전처리 없이 그대로 넣은 것보다 더 좋지 않은 점수 기록,,;;
# khaii로 전처리 하지 않는게 더 나을지도..?

In [101]:
# 입력된 문장 간 유사도 빠르게 구해보기
doc1 = '다만, 도로와 인접해서 거리의 소음이 들려요.'
doc2 = '하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다'
doc1_embedding = model.encode(doc1)
doc2_embedding = model.encode(doc2)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [102]:
# 입력 문장 - 문장 후보군 간 코사인 유사도 계산
cos_scores = util.pytorch_cos_sim(doc1_embedding, doc2_embedding)[0]
cos_scores

tensor([0.8280])

In [96]:
test_samples[3].texts, test_samples[3].label

(['다만, 도로와 인접해서 거리의 소음이 들려요.', '하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.'], 0.74)